# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
from IPython.display import display
import re 

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @datasci55696539

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [10]:
#Produto escolhido:
produto = 'era uma vez em hollywood -filter:retweets'

#Quantidade mínima de mensagens capturadas:
n = 750
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [11]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if not msg.retweeted:
        msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)


TweepError: Twitter error response: status code = 429

In [7]:
import re 

def cleanup(text):
    punctuation = '[!\-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [8]:
nariz = cleanup(msgs.lower())

AttributeError: 'list' object has no attribute 'lower'

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [ ]:
def cleanup(text):
    punctuation = '[!\"-.:?;,\n]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [ ]:
#filme = open("era uma vez em hollywood.xlsx", "r", encoding="utf8")
excel=pd.read_excel("era uma vez em hollywood.xlsx", sheet_name='Teste')

In [9]:
msgs

['rt @olhardigital: brad pitt confirmou, ao indiewire, que quentin tarantino pretende mesmo fazer uma versão estendida de era uma vez em holl…',
 'rt @vicebrasil: .\n\n         █          \u3000\u3000\n         █\u3000        \n         █\u3000        \n         █\u3000                           ＿\n   bacurau…',
 'rt @vicebrasil: .\n\n         █          \u3000\u3000\n         █\u3000        \n         █\u3000        \n         █\u3000                           ＿\n   bacurau…',
 'o tal charlie dos hippies (menção) do era uma vez em hollywood era o charles manson???',
 'rt @vicebrasil: .\n\n         █          \u3000\u3000\n         █\u3000        \n         █\u3000        \n         █\u3000                           ＿\n   bacurau…',
 'rt @leon_sem_ardo: impacto cultural:\n\n         █          \u3000\u3000\n         █\u3000        \n         █\u3000        \n         █…',
 'rt @leon_sem_ardo: impacto cultural:\n\n         █          \u3000\u3000\n         █\u3000        \n         █\u3

In [ ]:
ids = set()
for tweet in tweepy.Cursor(api.search, 
                    q="docker", 
                    Since="2016-08-09", 
                    #until="2014-02-15", 
                    lang="en").items(5000000):
if not tweet['retweeted'] and 'RT @' not in tweet['text']:
    #Write a row to the csv file/ I use encode utf-8
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.favorite_count, tweet.retweet_count, tweet.id, tweet.user.screen_name])
    #print "...%s tweets downloaded so far" % (len(tweet.id))
    ids.add(tweet.id) # add new id
    print ("number of unique ids seen so far: {}",format(len(ids)))
csvFile.close()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
serie_filme = pd.Series(excel.split())

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**